In [140]:
import numpy as np
import pandas as pd
from tensorflow import keras

In [141]:
# 파일 불러오기
train_csv = pd.read_csv('../mnist/train.csv',header=None)
train_csv.head(2)
tk_csv = pd.read_csv('../mnist/t10k.csv',header=None)
tk_csv.head(2)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


keras 모델로 구현하시오

In [142]:
# 데이터 전처리
train_input = train_csv.iloc[:,1:]  #input은 1열 삭제
test_input = tk_csv.drop(0,axis=1)

train_input.columns

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       775, 776, 777, 778, 779, 780, 781, 782, 783, 784],
      dtype='int64', length=784)

In [143]:
# target 정답 데이터 생성
train_target = train_csv[0]  # target은 1열
test_target = tk_csv[0]      

In [144]:
# input 데이터가 0~1값을 가지도록 표준화
train_scaled = train_input/256.0
test_scaled = test_input/256.0

print(train_scaled.shape, train_target.shape)

(1001, 784) (1001,)


In [145]:
print(train_target.value_counts().sort_values())

0
8     87
5     92
3     93
6     94
0     98
2     99
9    100
4    105
1    116
7    117
Name: count, dtype: int64


In [146]:
# 딥러닝
from sklearn.model_selection import train_test_split

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [147]:
np.unique(train_target)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [148]:
# 딥러닝 훈련
# 1.dense 밀집층
dense = keras.layers.Dense(10,activation='softmax',input_shape=(784,))

c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [149]:
# 2. 모델경정
model = keras.Sequential([dense])

# 3. 모델설정부분
# loss : 다중분류 결과값을 원핫인코딩을 사용하지 않으면 sparse_categorical_crossentropy
# loss : 다중분류 결과값을 원핫인코딩을 사용하면 categorical_crossentropy
# 이진 분류 : 원핫인코딩을 사용하지 않으면 sparse_binary_crossentropy
# 이진 분류 : 원핫인코딩을 사용하면 binary_crossentropy
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [150]:
# 4. 모델 훈련
# 반복 횟수 : epochs
# batch_size : 데이터를 뽑는 개수 32,64,128,... 32가 디폴트값. 안쓰면 32
model.fit(train_scaled,train_target,epochs=5)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2815 - loss: 2.1335   
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - accuracy: 0.7151 - loss: 1.4392
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8063 - loss: 1.0899 
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8157 - loss: 0.8991 
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8428 - loss: 0.7255 


In [151]:
# 5. 모델 평가
model.evaluate(val_scaled,val_target)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8624 - loss: 0.7036  


[0.7390536665916443, 0.8308457732200623]